# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: GPT-2
* Evaluation approach: evaluate method with a Hugging Face Trainer
* Fine-tuning dataset: dair-ai/emotion

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 61.3 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 35.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding
from datasets import load_dataset
from peft import LoraConfig, PeftModelForSequenceClassification, TaskType, AutoPeftModelForSequenceClassification
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [3]:
# load the dataset
dataset = load_dataset("dair-ai/emotion")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
train_dataset = dataset['train']
val_dataset = dataset['validation']

In [5]:
len(train_dataset)

16000

In [6]:
len(val_dataset)

2000

In [7]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [8]:
train_dataset[0]

{'text': 'i didnt feel humiliated', 'label': 0}

In [9]:
val_dataset[10]

{'text': 'i feel bitchy but not defeated yet', 'label': 3}

In [10]:
emotion_to_id_dict = {
    "sadness": 0, "joy": 1, "love": 2, "anger": 3, "fear": 4, "surprise": 5
}
id_to_emotion_dict = {
    0: "sadness", 1: "joy", 2: "love", 3: "anger", 4: "fear", 5: "surprise"
}

In [11]:
# the tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
# Tokenize and convert
def tokenize_and_encode(examples):
    tokenized_inputs = tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

train_dataset = train_dataset.map(tokenize_and_encode, batched=True)
val_dataset = val_dataset.map(tokenize_and_encode, batched=True)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [13]:
# model
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=len(emotion_to_id_dict.keys()))
model.config.pad_token_id = tokenizer.pad_token_id

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
# Compute metrics function
def compute_metrics(eval_pred):
    preds = np.argmax(eval_pred.predictions, axis=1)
    precision, recall, f1_score, _ = precision_recall_fscore_support(eval_pred.label_ids, preds, average="weighted")
    return {"accuracy": accuracy_score(eval_pred.label_ids, preds), "f1_score": f1_score, "precision": precision, "recall": recall}

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_ratio=0.1,
)

# initialize the Trainer with compute_metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Evaluate
evaluation_results = trainer.evaluate()
print("Evaluation Results:", evaluation_results)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Evaluation Results: {'eval_loss': 4.8967976570129395, 'eval_accuracy': 0.275, 'eval_f1_score': 0.11867398979992154, 'eval_precision': 0.07566283141570786, 'eval_recall': 0.275, 'eval_runtime': 72.9037, 'eval_samples_per_second': 27.433, 'eval_steps_per_second': 6.858}


/home/student/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [16]:
# model backbone network
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=6, bias=False)
)

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [17]:
# PEFT model configuration
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=4,
    lora_alpha=16,
    lora_dropout=0.1
)

# load the pre-trained GPT-2 model
model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=len(emotion_to_id_dict.keys()))
model.config.pad_token_id = model.config.eos_token_id

# peft model                                                           
peft_model = PeftModelForSequenceClassification(model, peft_config)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [18]:
peft_model.print_trainable_parameters()

trainable params: 156,672 || all params: 124,596,480 || trainable%: 0.12574352020217586


In [19]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results/peft_model",
    evaluation_strategy="epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs/peft_model',
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    warmup_ratio=0.1,
)

# initialize the Trainer with compute_metrics
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [20]:
# start the training
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1 Score,Precision,Recall
1,0.371500,0.337471,0.921500,0.921083,0.921547,0.921500


TrainOutput(global_step=4000, training_loss=0.7070492005348206, metrics={'train_runtime': 1461.0126, 'train_samples_per_second': 10.951, 'train_steps_per_second': 2.738, 'total_flos': 4188373254144000.0, 'train_loss': 0.7070492005348206, 'epoch': 1.0})

In [21]:
# Evaluate
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.3374706208705902, 'eval_accuracy': 0.9215, 'eval_f1_score': 0.9210827282146451, 'eval_precision': 0.9215473462323768, 'eval_recall': 0.9215, 'eval_runtime': 77.9521, 'eval_samples_per_second': 25.657, 'eval_steps_per_second': 6.414, 'epoch': 1.0}


In [22]:
peft_model.save_pretrained('model/peft_model')

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [23]:
inference = AutoPeftModelForSequenceClassification.from_pretrained(
    "model/peft_model",
    num_labels=len(emotion_to_id_dict.keys())
)
inference.config.pad_token_id = inference.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
trainer = Trainer(
    model=inference,
    args=training_args,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# model evaluation
results = trainer.evaluate()
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 0.3374706208705902, 'eval_accuracy': 0.9215, 'eval_f1_score': 0.9210827282146451, 'eval_precision': 0.9215473462323768, 'eval_recall': 0.9215, 'eval_runtime': 77.593, 'eval_samples_per_second': 25.776, 'eval_steps_per_second': 6.444}


In [27]:
import torch

def prediction(sentence: str) -> str:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inference.to(device)

    # input text
    inputs = tokenizer(sentence, return_tensors="pt").to(device)

    # predictions
    with torch.no_grad():
        outputs = inference(**inputs)
        logits = outputs.logits

    probabilities = torch.nn.functional.softmax(logits, dim=1)
    predicted_class = probabilities.argmax().item()
    predicted_emotion = id_to_emotion_dict[predicted_class]

    return (predicted_class, predicted_emotion)

# inference
text = "I am so sad"
predicted_result = prediction(text)
print(f"Sentence: '{text}'\nPredicted label: {predicted_result[0]}\nPredicted emotion: {predicted_result[1]}")

Sentence: 'I am so sad'
Predicted label: 0
Predicted emotion: sadness


In [28]:
text = "wow, I didn't expect that"
predicted_result = prediction(text)
print(f"Sentence: '{text}'\nPredicted label: {predicted_result[0]}\nPredicted emotion: {predicted_result[1]}")

Sentence: 'wow, I didn't expect that'
Predicted label: 1
Predicted emotion: joy


##### Peft model seems to perform a little better compared to the foundational model as we can see the accuracy increses by ~65% (27% to 92%) with significant increase in f1 score and decrease in the validation loss.